## Importing the libraries

In [914]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

## Importing the dataset

In [915]:
dataset = pd.read_excel("dataset.xlsx")
dataset.head()

,KOD,BARKOD,MAMUL,K.İÇİ,GR,LİSTE,FATURA ALTI %,DÖNEM SONU %,KDV,NET MALİYET,Ö.RAF,KATEGORİ,AY
0,67293883,8690637840746,1,8,400.0,21.85,12.0,5,0.01,18.449266,23.061583,FOOD,MART
1,67293891,8690637840821,1,8,400.0,21.85,12.0,5,0.01,18.449266,23.061583,FOOD,MART
2,67767533,8690637905896,1,12,600.0,27.00,13.0,5,0.01,22.538655,28.173319,FOOD,MART
3,67754288,8690637905193,1,12,600.0,27.00,13.0,5,0.01,22.538655,28.173319,FOOD,MART
4,68612788,8690637999277,1,8,335.0,39.75,10.0,5,0.01,34.326113,42.907641,FOOD,MART


In [916]:
selected_columns = ['KATEGORİ', 'AY', 'MAMUL', 'K.İÇİ', 'GR', 'LİSTE', 'NET MALİYET']
data_selected = dataset[selected_columns]
data_selected.head()

,KATEGORİ,AY,MAMUL,K.İÇİ,GR,LİSTE,NET MALİYET
0,FOOD,MART,1,8,400.0,21.85,18.449266
1,FOOD,MART,1,8,400.0,21.85,18.449266
2,FOOD,MART,1,12,600.0,27.00,22.538655
3,FOOD,MART,1,12,600.0,27.00,22.538655
4,FOOD,MART,1,8,335.0,39.75,34.326113


In [917]:
X = data_selected.drop(columns=["KATEGORİ"])  # Bağımsız değişkenler
y = data_selected["KATEGORİ"]  # Bağımlı değişken

In [918]:
X.head()

,AY,MAMUL,K.İÇİ,GR,LİSTE,NET MALİYET
0,MART,1,8,400.0,21.85,18.449266
1,MART,1,8,400.0,21.85,18.449266
2,MART,1,12,600.0,27.00,22.538655
3,MART,1,12,600.0,27.00,22.538655
4,MART,1,8,335.0,39.75,34.326113


## Encoding categorical data

### Encoding the Dependent Variable

In [919]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

In [920]:
le = LabelEncoder()
y = le.fit_transform(y)

### Encoding the Independent Variable

In [921]:
categorical_columns = ['AY', 'MAMUL']
numeric_columns = ['K.İÇİ', 'GR', 'LİSTE', 'NET MALİYET']

In [922]:
preprocessor = ColumnTransformer(
    transformers=[
        ("sayısal", StandardScaler(), numeric_columns),
        ("kategori", OneHotEncoder(), categorical_columns),
    ],
    remainder="passthrough",
)

X = preprocessor.fit_transform(X)


In [923]:
x_dataset = pd.DataFrame(X.toarray())
x_dataset.head(1)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,-0.565928,-0.110449,-0.986225,-0.971577,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [924]:
from sklearn.model_selection import train_test_split

# Verileri eğitim ve test kümelerine bölün
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [925]:
X_test[0].toarray()[0]

array([-0.61347256,  1.38842641,  1.56352527,  1.61853602,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

#### Modeli oluşturun

In [926]:
from tensorflow import keras
from keras import layers 

# yapay sinir ağı modeli
model = keras.Sequential([ # Sıralı bir model
    layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]]), # Giriş katmanı
    layers.Dense(64, activation='relu'), # Gizli katman
    layers.Dense(3, activation='softmax') # Çıkış katmanı
])



#### Modeli Derle

In [927]:
model.compile(
    optimizer='adam', # Optimizasyon fonksiyonu
    loss='sparse_categorical_crossentropy', # Kayıp fonksiyonu
    metrics=['accuracy'] # Başarı metriği
)


#### Eğitim işlemini başlatın

In [928]:
X_train = X_train.toarray()
y_train = y_train.reshape(-1, 1)

In [929]:

history = model.fit(
    X_train, y_train, # Eğitim verileri
    validation_split=0.6, # Eğitim verilerinin %60'sini doğrulama verileri olarak kullanın
    batch_size=32, # Aynı anda işlenecek örnek sayısı
    epochs=35, # 35 iterasyon (döngü)
    verbose=1, # Her iterasyonda ekrana çıktı ver (1: evet, 0: hayır)
)


Epoch 1/35


39/39 [==============================] - 0s 4ms/step - loss: 0.8129 - accuracy: 0.7561 - val_loss: 0.5636 - val_accuracy: 0.8762
Epoch 2/35
39/39 [==============================] - 0s 2ms/step - loss: 0.3529 - accuracy: 0.9444 - val_loss: 0.1949 - val_accuracy: 0.9760
Epoch 3/35
39/39 [==============================] - 0s 2ms/step - loss: 0.1039 - accuracy: 0.9935 - val_loss: 0.0605 - val_accuracy: 0.9945
Epoch 4/35
39/39 [==============================] - 0s 2ms/step - loss: 0.0349 - accuracy: 0.9984 - val_loss: 0.0260 - val_accuracy: 0.9973
Epoch 5/35
39/39 [==============================] - 0s 2ms/step - loss: 0.0158 - accuracy: 1.0000 - val_loss: 0.0145 - val_accuracy: 1.0000
Epoch 6/35
39/39 [==============================] - 0s 2ms/step - loss: 0.0090 - accuracy: 1.0000 - val_loss: 0.0090 - val_accuracy: 1.0000
Epoch 7/35
39/39 [==============================] - 0s 2ms/step - loss: 0.0057 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 8/35
39/39 [=============

#### Eğitim ve doğrulama kayıp grafiği

In [930]:
print(history.history.keys())
# dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])
# loss: eğitim verileri için kayıp fonksiyonu
# accuracy: eğitim verileri için başarı metriği
# val_loss: doğrulama verileri için kayıp fonksiyonu
# val_accuracy: doğrulama verileri için başarı metriği  


dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


In [935]:
# Eğitim ve doğrulama kayıp grafiği ve Eğitim ve doğrulama başarı grafiği

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(y=history.history['loss'], name="Eğitim Kaybı"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=history.history['val_loss'], name="Doğrulama Kaybı"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(y=history.history['accuracy'], name="Eğitim Başarısı"),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(y=history.history['val_accuracy'], name="Doğrulama Başarısı"),
    row=1, col=2
)

fig.update_layout(height=500, width=1000, title_text="Eğitim ve Doğrulama Kayıp ve Başarı Grafiği")
fig.show()



In [932]:
X_test = X_test.toarray()
y_test = y_test.reshape(-1, 1)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

 1/24 [>.............................] - ETA: 0s - loss: 8.5781e-05 - accuracy: 1.0000

24/24 [==============================] - 0s 739us/step - loss: 1.5760e-04 - accuracy: 1.0000


### Test verileri üzerinde tahmin 


In [933]:
# Test verileri üzerinde tahmin 
y_pred = model.predict(X_test) # Tahminlerin olasılık değerleri
y_pred = np.argmax(y_pred, axis=1) # En yüksek olasılığa sahip sınıfı tahmin olarak seçildi
y_pred = le.inverse_transform(y_pred) # Tahminlerin sınıf isimlerine dönüştürülmesi
y_pred = pd.Series(y_pred, name="Tahmin") # Tahminlerin bir pandas serisine dönüştürülmesi

# Test verileri üzerinde gerçek değerler
y_test = le.inverse_transform(y_test.reshape(-1)) # Gerçek değerlerin sınıf isimlerine dönüştürülmesi
y_test = pd.Series(y_test, name="Gerçek Değer") # Gerçek değerlerin bir pandas serisine dönüştürülmesi

# Tahmin ve gerçek değerleri birleştirin
result = pd.concat([y_test, y_pred], axis=1) # axis=1: sütun bazında birleştirme yapılması

# test sonuçları


24/24 [==============================] - 0s 522us/step


In [934]:
from sklearn.metrics import f1_score

print("Eğitim Yineleme sayısı: ", history.params['epochs'])
print("Eğitim doğruluk oranı: ", history.history['accuracy'][-1])
print("Test doğruluk oranı: ", test_acc)
print("Test kaybı: ", test_loss)
print("f1 score: ", f1_score(y_test, y_pred, average='weighted'))

Eğitim Yineleme sayısı:  35
Eğitim doğruluk oranı:  1.0
Test doğruluk oranı:  1.0
Test kaybı:  0.0001576020586071536
f1 score:  1.0


In [938]:
# karışıklık matrisi
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm = pd.DataFrame(cm, columns=le.classes_, index=le.classes_)
cm.index.name = "Gerçek"
cm.columns.name = "Tahmin"
cm

# görselleştirme
import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(
    z=cm.values,
    x=list(cm.columns),
    y=list(cm.index),
    colorscale='Viridis',
    showscale=True,
    reversescale=True,
)

fig.update_layout(
    title_text="Karışıklık Matrisi",
    xaxis = dict(title="Tahmin"),
    yaxis = dict(title="Gerçek")
)

fig.show()



              precision    recall  f1-score   support

        FOOD       1.00      1.00      1.00       269
          HC       1.00      1.00      1.00       171
          PC       1.00      1.00      1.00       324

    accuracy                           1.00       764
   macro avg       1.00      1.00      1.00       764
weighted avg       1.00      1.00      1.00       764

